<a href="https://colab.research.google.com/github/shellgav/Analisis_ventas/blob/main/ModeloP2d0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures


# **Carga la base de datos limpia y pegada, lista para el analisis**

In [ ]:
df_join = pd.read_csv('df_joinR.csv')
df_join.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Week,Year,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.00,0.0,0.00,0.00,0.00,211.096358,8.106,1.0,2010.0,A,151315
1,1,1,2010-02-05,24924.50,False,55.32,3.386,6766.44,5147.7,50.82,3639.90,2737.42,223.462779,6.573,1.0,2010.0,A,151315
2,1,1,2010-02-12,46039.49,True,38.51,2.548,0.00,0.0,0.00,0.00,0.00,211.242170,8.106,2.0,2010.0,A,151315
3,1,1,2010-02-19,41595.55,False,39.93,2.514,0.00,0.0,0.00,0.00,0.00,211.289143,8.106,3.0,2010.0,A,151315
4,1,1,2010-02-19,41595.55,False,52.92,3.252,9696.28,292.1,103.78,1133.15,6612.69,223.512911,6.573,3.0,2010.0,A,151315


In [ ]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435734 entries, 0 to 435733
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         435734 non-null  int64  
 1   Dept          435734 non-null  int64  
 2   Date          435734 non-null  object 
 3   Weekly_Sales  435734 non-null  float64
 4   IsHoliday     435734 non-null  bool   
 5   Temperature   435734 non-null  float64
 6   Fuel_Price    435734 non-null  float64
 7   MarkDown1     435734 non-null  float64
 8   MarkDown2     435734 non-null  float64
 9   MarkDown3     435734 non-null  float64
 10  MarkDown4     435734 non-null  float64
 11  MarkDown5     435734 non-null  float64
 12  CPI           435734 non-null  float64
 13  Unemployment  435734 non-null  float64
 14  Week          435734 non-null  float64
 15  Year          435734 non-null  float64
 16  Type          435734 non-null  object 
 17  Size          435734 non-null  int64  
dtypes: b

In [ ]:
df_join = df_join.drop("Date", axis = 1)
df_join = df_join.drop("Year", axis = 1) # se elimina la columna date para evitar redundanciasy multicolinealidad

In [ ]:
df_join.head()

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Week,Type,Size
0,1,1,24924.50,False,42.31,2.572,0.00,0.0,0.00,0.00,0.00,211.096358,8.106,1.0,A,151315
1,1,1,24924.50,False,55.32,3.386,6766.44,5147.7,50.82,3639.90,2737.42,223.462779,6.573,1.0,A,151315
2,1,1,46039.49,True,38.51,2.548,0.00,0.0,0.00,0.00,0.00,211.242170,8.106,2.0,A,151315
3,1,1,41595.55,False,39.93,2.514,0.00,0.0,0.00,0.00,0.00,211.289143,8.106,3.0,A,151315
4,1,1,41595.55,False,52.92,3.252,9696.28,292.1,103.78,1133.15,6612.69,223.512911,6.573,3.0,A,151315


In [ ]:
df_join = pd.get_dummies(df_join, columns=['Type'], drop_first=True)
df_join.head()

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Week,Size,Type_B,Type_C
0,1,1,24924.50,False,42.31,2.572,0.00,0.0,0.00,0.00,0.00,211.096358,8.106,1.0,151315,False,False
1,1,1,24924.50,False,55.32,3.386,6766.44,5147.7,50.82,3639.90,2737.42,223.462779,6.573,1.0,151315,False,False
2,1,1,46039.49,True,38.51,2.548,0.00,0.0,0.00,0.00,0.00,211.242170,8.106,2.0,151315,False,False
3,1,1,41595.55,False,39.93,2.514,0.00,0.0,0.00,0.00,0.00,211.289143,8.106,3.0,151315,False,False
4,1,1,41595.55,False,52.92,3.252,9696.28,292.1,103.78,1133.15,6612.69,223.512911,6.573,3.0,151315,False,False


In [ ]:
df_join1 = df_join.groupby(['Week','Store']).agg({    'Weekly_Sales': 'sum', 'Temperature': 'mean',
    'Fuel_Price': 'mean',
    'CPI': 'mean',
    'Unemployment': 'mean',
    'MarkDown1': 'sum',
    'MarkDown2': 'sum',
    'MarkDown3': 'sum',
    'MarkDown4': 'sum',
    'MarkDown5': 'sum',
    'IsHoliday': 'max',
    'Type_B': 'max',
    'Type_C': 'max',
    'Size':'mean', }).reset_index()
# Agrupamos y agregamos los datos por semana y tienda para obtener una tabla en un formato más adecuado para el análisis
# y la predicción de modelos. Esto permite resumir las ventas y otras variables clave, facilitando la identificación
#de patrones y tendencias relevantes para el modelo

In [ ]:
y  = df_join1["Weekly_Sales"] # Variable Dependiente
X = df_join1.drop("Weekly_Sales", axis = 1) # Variables Independiente

In [ ]:
# División del Dataset en Conjunto de Entrenamiento y Prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Se crea el modelo de regresión
modelo_lineal = LinearRegression()
modelo_lineal


LinearRegression()

In [ ]:
modelo_lineal.fit(X_train, y_train)

LinearRegression()

In [ ]:
# poner a pruba el modelo creado
y_pred = modelo_lineal.predict(X_test)

In [ ]:
# Mean squared error (MSE )
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# R2
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

Mean Squared Error: 352020005974.35266
Root Mean Squared Error: 593312.7387595455
Mean Absolute Error: 393995.35232289287
R2 Score: 0.5027391591364971


***Prueba del modelo Lineal con el 100% de los datos***


In [ ]:
# División del Dataset en Conjunto de Entrenamiento y Prueba y se crea el modelo
X_train, X_test, y_train, y_test = train_test_split(X, y)
modelo_lineal = LinearRegression()
modelo_lineal.fit(X_train, y_train)



LinearRegression()

In [ ]:
y_pred_train = modelo_lineal.predict(X_train)
print(y_pred_train[:5])

[2045721.52024587  378628.20273793  352736.62206445  145305.1315862
 1904225.84526757]


In [ ]:
# poner a pruba el modelo creado
y_pred = modelo_lineal.predict(X_test)

In [ ]:
# Mean squared error (MSE )
print (f'MSE: {mean_squared_error(y_train, y_pred_train)}')

# RMSE
print (f'RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train))}')

# MAE
print (f'MAE: {mean_absolute_error(y_train, y_pred_train)}')

# R2
print (f'R2: {r2_score(y_train, y_pred_train)}')

MSE: 278475822030.5641
RMSE: 527708.0841057526
MAE: 367341.78385746235
R2: 0.5582535802723798


# **Modelo Polinomico**


In [ ]:
y  = df_join["Weekly_Sales"] # Variable Dependiente
X = df_join.drop("Weekly_Sales", axis = 1) # Variables Independiente
# Generar características polinómicas
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Dividir las características polinómicas
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)

# Entrenar el modelo
lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_train_poly, y_train)

# Hacer predicciones
y_pred_poly = lin_reg_poly.predict(X_test_poly)

# Evaluar el modelo
r2_poly = r2_score(y_test, y_pred_poly)
mse_poly = mean_squared_error(y_test, y_pred_poly)
mae_poly = mean_absolute_error(y_test, y_pred_poly)

print(f'R^2 Polinómico: {r2_poly}')
print(f'Error Cuadrático Medio Polinómico: {mse_poly}')
print(f'Error Absoluto Medio Polinómico: {mae_poly}')

R^2 Polinómico: 0.229419428605373
Error Cuadrático Medio Polinómico: 389976412.5166637
Error Absoluto Medio Polinómico: 12560.794017733271
